# Scraper Running


#### Also, this was the good one.  The one that actually scraped.

In [1]:
# Scraping function built in a previous notebook and saved to a local file
from scrape_lib import connect_to_page
from scrape_lib import get_liquor_list

In [2]:
import pandas as pd


The `connect_to_page` function is for utilizing the selenium headless browser.  
The `get_liquor_list` function acquires all the URLs for individual scotches that I'll use later to go to each page. 

In [2]:
from bs4 import BeautifulSoup
import requests
import re
import urllib

import pandas as pd
from IPython.display import clear_output

In [3]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from scrapy.selector import Selector
import re
import time
import csv
from itertools import islice

In [2]:
# get all individual scotch URLs
scotch_home = 'https://www.totalwine.com/spirits/scotch/c/000885?tab=fullcatalog&sort=name-asc&spiritsvolume=standard-size-750-ml&pagesize=180'
scotch = get_liquor_list(scotch_home, 8, head_less= True)

KeyboardInterrupt: 

In [6]:
len(scotch)

1440

In [5]:
len(set(scotch))

1440

In [7]:
# TEQUILA!!!
# All individual Tequila URLs
tequila_home = 'https://www.totalwine.com/spirits/tequila/c/000824?tab=fullcatalog&text=&spiritsvolume=standard-size-750-ml&pagesize=180'
tequila = get_liquor_list(tequila_home, 5, head_less= True)

In [8]:
print(len(tequila))
print(len(set(tequila)))

886
886


Selenium works like any other web browser in that once I have a window open, I can use it to connect to other pages and jump around. So first I need to launch the browser and connect to the main page before I start browsing all the individual pages.

Total Wine has two different ways of asking if you're over 21.  One has a blue button and one has a red button so I made options to close both depending on which actually pops up.

The below tests the connecting to the page and navigating to a new one. (This code is implemented later on in the function `scrape_liquor_list`.

In [9]:
starting = 'https://www.totalwine.com/spirits/scotch/c/000885'
driver = connect_to_page(starting, headless=False)
time.sleep(1)
load = False
# Total wine appears to have two different options that ask for age
while load == False:
    try:
        # Option 1 : Red Button
        red_age = '//*[@id="btnYes"]'
        driver.find_element_by_xpath(red_age).click()
        load = True
    except:
        pass

    try:
        # Option 2 : Blue Button
        blue_age = '/html/body/div[2]/div/div[2]/div[3]/div/button[2]'
        driver.find_element_by_xpath(blue_age).click()
        load = True
    except:
        pass

    time.sleep(0.5)

    # if it prompts for online or instore via popup.
try:
    shop_button = '/html/body/header/div[3]/div/div[1]'
    driver.find_element_by_xpath(shop_button).click()

except:
    pass

### ----------------------------------------------------------------

driver.get('https://www.totalwine.com/spirits/scotch/single-malt/a-d-rattray-aberfeldy-18-yr/p/134739750?s=1108&igrules=true')

#driver.get('https://www.totalwine.com/spirits/scotch/blended-scotch/johnnie-walker-black-label/p/636175?s=2301&igrules=true')


load_more = True
while load_more == True:
    try:
        load_button = '//*[@id="BVRRContainer"]/div/div/div/div/div[3]/div/button'
        driver.find_element_by_xpath(load_button).click()
        time.sleep(1)
    except:
        load_more = False
#page = BeautifulSoup(driver.page_source, 'lxml')


In [10]:
driver.quit()

In [20]:
# FUnction to get the user reviews
def aquire_reviews(liquor_page, no_reviews):
    if no_reviews > 0:
        reviews_list = '//*[@id="BVRRContainer"]/div/div/div/div/ol'
        review_section = Selector(text=liquor_page.page_source).xpath(reviews_list).extract()


        soup_reviews = BeautifulSoup(review_section[0], 'lxml')


        reviews_dict = {}
        itera = 0

        for review in soup_reviews.find_all('div',
            {'class':'bv-content-item-author-profile-offset bv-content-item-author-profile-offset-on'})[:-1]:

            
            review_desc = review.find('h4', {'class':'bv-content-title'}).text
            review_cont = review.find('div',{'class':'bv-content-summary-body-text'}).text
            review_scor = review.find('span',{'class':'bv-off-screen'}).text[0]

            review = {'review_desc':review_desc,
                      'review_cont':review_cont,
                      'review_scor':review_scor}

            reviews_dict['review_'+str(len(reviews_dict)+1)] = review
        # I had this return statement tabbed over so it only got one.
        return(reviews_dict)

    else:
        return 'No Reviews'
    
    

#print(aquire_reviews(html))


In [21]:
# create the empty DF to populate.
driver = connect_to_page('https://www.totalwine.com/spirits/scotch/c/000885')

df = pd.DataFrame(None, columns = ['title','price','size','avg_rating',
                                   'taste','score','description','brand',
                                  'country','state','spirit_type','spirit_style',
                                  'reviews'])

driver.quit()
#bottle_scraper(driver, )

In [17]:
driver.quit()

In [22]:
# Scrape Individual Page

def bottle_scraper(driver, url, output_df , output_sql = None):
    driver.get(url)
    
    load_more = True
    while load_more == True:
        try:
            load_button = '//*[@id="BVRRContainer"]/div/div/div/div/div[3]/div/button'
            driver.find_element_by_xpath(load_button).click()
            time.sleep(1)
        except:
            load_more = False
    page = BeautifulSoup(driver.page_source, 'lxml')
    
    #review number
    try:
        num_reviews = int(page.find('div',{'class':'bv_numReviews_component_container'}).find('meta')['content'])
    except:
        num_reviews = 0
    
    # Item Info
    title = page.find('h1', {'class':'product-name'}).find('input')['value']
    price = page.find('input', {'id':"anPRice"})['value']
    size = page.find('span', {'class':"single-bottle bottle-options"}).text
    
    if page.find('div', {'class':"bv_avgRating_component_container"}):
        avg_rating = page.find('div', {'class':"bv_avgRating_component_container"}).text
    else:
        avg_rating = -1
        
    # Item Overview
    if page.find(name = 'div', attrs = {'id':"product-details-section"}):
        overview = page.find(name = 'div', attrs = {'id':"product-details-section"})
        if overview.find('div' , {'class':"bottom-full-desc"}):
            overview_taste = overview.find('div' , {'class':"bottom-full-desc"}).text
        else:
            overview_taste = 'NA'
        if overview.find('div', {'class':'right-full-desc'}):
            overview_desc = overview.find('div', {'class':'right-full-desc'}).text
        else:
            overview_desc = 'NA'
        if overview.find('span', {'class':'nine-six-logo'}):
            overview_score = overview.find('span', {'class':'nine-six-logo'}).text
        else:
            overview_score = -1
        
    if page.find('main', {'class': "main pdp an-productDetails"}).find('meta')['content']:
        brand = page.find('main', {'class': "main pdp an-productDetails"}).find('meta')['content']
    else:
        brand = 'NA'
    # Item Details
    if page.find('ul', {'class' : "wine-right-details"}):
        details = page.find('ul', {'class' : "wine-right-details"})
    else:
        details = 'NA'
    if details.find('a',{'class':'analyticsCountryState'}):
        country = details.find('a',{'class':'analyticsCountryState'}).text
    else:
        country = 'NA'
    if details.find('a',{'class':'analyticsRegion'}):
        state = details.find('a',{'class':'analyticsRegion'}).text
    else:
        state = 'NA'
    if details.find('a',{'class':'analyticsProductType'}):
        spirit_type = details.find('a',{'class':'analyticsProductType'}).text
    else:
        spirit_type = 'NA'
    if details.find('a',{'class':'analyticsVarietalType'}):
        spirit_style = details.find('a',{'class':'analyticsVarietalType'}).text
    else:
        spirit_style = 'NA'
    
    
    reviews_dict = aquire_reviews(driver, num_reviews)
    
    row = [title, price, size, avg_rating, overview_taste, overview_score, overview_desc, brand,
          country, state, spirit_type, spirit_style, reviews_dict]
    
    output_df.loc[len(output_df)] = row
    
    if output_sql:
        # establish SQL engine.
        pass


In [23]:
def scrape_liquor_list(starting_page,  liquor_list, head_less = True, output_as_df = True, output_as_sql = False):
    driver = connect_to_page(starting_page, headless=head_less)
    time.sleep(1)
    load = False
    # Total wine appears to have two different options that ask for age
    while load == False:
        try:
            # Option 1 : Red Button
            red_age = '//*[@id="btnYes"]'
            driver.find_element_by_xpath(red_age).click()
            load = True
        except:
            pass

        try:
            # Option 2 : Blue Button
            blue_age = '/html/body/div[2]/div/div[2]/div[3]/div/button[2]'
            driver.find_element_by_xpath(blue_age).click()
            load = True
        except:
            pass

        time.sleep(0.5)
    
        # if it prompts for online or instore via popup.
    try:
        shop_button = '/html/body/header/div[3]/div/div[1]'
        driver.find_element_by_xpath(shop_button).click()

    except:
        pass
        
    if output_as_df:
        df = pd.DataFrame(None, columns = ['title','price','size','avg_rating',
                                           'taste','score','description','brand',
                                          'country','state','spirit_type','spirit_style',
                                          'reviews'])
    # Now that the pop-ups have been clicked we can iterate through the pass
    bottle_num = 1
    for bottle_url in liquor_list:
        clear_output()
        print(bottle_num, bottle_url,)
        bottle_scraper(driver, bottle_url, df , output_sql = None)
        bottle_num +=1
    driver.quit()
    return df

In [ ]:
driver.quit()

In [14]:
test_list = ['https://www.totalwine.com/spirits/scotch/blended-scotch/johnnie-walker-black-label/p/636175?s=2301&igrules=true',
            'https://www.totalwine.com/spirits/scotch/single-malt/a-d-rattray-aberfeldy-18-yr/p/134739750?s=1108&igrules=true',
            'https://www.totalwine.com/spirits/scotch/single-malt/lagavulin-16-yr/p/643750?s=2301&igrules=true']

In [27]:

scotches = scrape_liquor_list(starting_page='https://www.totalwine.com/spirits/scotch/c/000885', 
                   liquor_list= test_list,
                  head_less = True,
                  output_as_df = True,
                  output_as_sql = False)

3 https://www.totalwine.com/spirits/scotch/single-malt/lagavulin-16-yr/p/643750?s=2301&igrules=true


In [32]:
scotches['reviews'][2]

{'review_1': {'review_cont': ' Just remove the cork and you will understand. I am new to whisky and this bottle blew my mind. So much more body and complexity than other bottles in my bar. If you are brave and adventurouse, Lagavulin 16 is a must. ',
  'review_desc': '    Holy smokes!   ',
  'review_scor': '5'},
 'review_2': {'review_cont': ' I love scotch with heavy peat and smoke.  Lagavulin 16 has it all. ',
  'review_desc': '    Smoking . . . .   ',
  'review_scor': '5'}}

Only getting a max of 2 reviews now...

In [ ]:
scotches = scrape_liquor_list(starting_page='https://www.totalwine.com/spirits/scotch/c/000885', 
                   liquor_list= scotch,
                  head_less = True,
                  output_as_df = True,
                  output_as_sql = False)

_Manually stopped the full scrape to end it cause it takes a long time, this was on the re-run to validate code._

In [164]:
scotches.to_csv('scotches_df.csv')

In [180]:
scotches[scotches['reviews'] != 'None']

,title,price,size,avg_rating,taste,score,description,brand,country,state,spirit_type,spirit_style,reviews
0,A.D. Rattray Aberfeldy 18 Yr,99.99,750ml,-1,,-1,"Highlands, Scotland- Pleasant aromas of caram...",A.D. Rattray,Scotland,Highland,Scotch,Single Malt,No Reviews
1,A.D. Rattray Auchroisk 22 Yr,139.99,750ml,-1,,-1,"Speyside, Scotland - This Auchroisk is quite ...",A.D. Rattray,Scotland,NA,Scotch,Single Malt,No Reviews
2,A.D. Rattray Aultmore 16 Year,97.99,750ml,-1,,-1,"Speyside, Scotland- Aroma of honey, ripe frui...",A.D. Rattray,Scotland,NA,Scotch,Single Malt,No Reviews
3,A.D. Rattray Ben Nevis 17Yr Single Cask Barrel...,99.99,750ml,-1,Delicate . Apple . Grass . Vanilla . Complex,-1,NA,A.D. Rattray,Scotland,Highland,Scotch,Single Malt,No Reviews
4,A.D. Rattray Benriach 23 Yr,179.99,750ml,-1,,-1,"Speyside, Scotland- Rich aromas of honey and ...",A.D. Rattray,Scotland,NA,Scotch,Single Malt,No Reviews
5,A.D. Rattray Blair Athol 22 Yr,149.99,750ml,-1,,-1,"Highlands, Scotland - Golden hues. To the nos...",A.D. Rattray,Scotland,Highland,Scotch,Single Malt,No Reviews
6,A.D. Rattray Cask Islay Single Malt,57.99,750ml,-1,,-1,"Islay, Scotland- This limited edition blended...",A.D. Rattray,Scotland,Islay & Islands,Scotch,Single Malt,No Reviews
7,A.D. Rattray Craigellachie 14 yr Barrel Select,109.99,750ml,-1,,-1,NA,Craigellachie,Scotland,Speyside,Scotch,Single Malt,No Reviews
8,A.D. Rattray Fettercairn 10Yr,76.99,750ml,-1,,-1,"Scotland- NOSE: Wine gums, coconut water, ora...",A.D. Rattray,Scotland,Highland,Scotch,Single Malt,No Reviews
9,A.D. Rattray Girvan 1964 50 Yr,899.99,750ml,-1,Rich . Orange . Chocolate . Nut . Long,-1,"Lowlands, Scotland - This 50 year old express...",A.D. Rattray,Scotland,Lowland,Scotch,Single Malt,No Reviews
